In [ ]:
to add to SQL cheat sheet 

lower
trim 

lower(trim(product_name))
date_format(sale_data, '%Y-%m')


(SELECT *,
 DENSE_RANK() OVER (PARTITION BY DepartmentID ORDER BY Salary DESC) AS DeptPayRank
 FROM Employee e
) cte


while queue and cond_met:
    # process at current level
    for n in range(len(queue)):
        val_old = queue.popleft()
        queue.append(val_new)

# verified order 
join
where
group by 
having 

        
HAVING 
GROUP BY 
JOIN
SUBQUERY 
CTE

FROM
ORDER BY
LIMIT 

LIMIT offset, row_count
LIMIT 5, 1 give the 5th value only


select 
player_id, 
device_id 
from 
(select player_id, 
        device_id, 
        rank() over (partition by player_id order by event_date ASC) as ranking
 from activity) cte
where cte.ranking = 1;

select 
player_id, 
device_id 
from 
activity 
where (player_id, event_date) in 
(select player_id,
        min(event_date)
        from activity 
        group by player_id) 

select 
player_id, 
device_id 
from activity 
where (player_id, event_date) in 
(select player_id,
        min(event_date)
        from activity 
        group by player_id) 

# order
from
join
where
order by

select
player_id,
device_id
from 
(select player_id, device_id, min(event_date) from Activity group by player_id) cte;

partition - sometimes can rewrite using case and groupby 

cumulative sum can be done w/ self join on s1.day <= s2.day 
# cumulative sum 
select
a1.player_id,
a1.event_date,
sum(a2.games_played) as games_played_so_far
from Activity a1
inner join Activity a2 on a1.player_id = a2.player_id
and a1.event_date >= a2.event_date
group by a1.player_id, a1.event_date

# max by project 
select 
p.project_id,
p.employee_id,
rank() over(partition by p.project_id order by e.experience_years desc) as ranking
from Project p join Employee e
on p.employee_id = e.employee_id

IF (expression_to_evaluate, result_if_true, result_if_false)

CASE - dont use parenthesis 

https://leetcode.com/problems/activity-participants/
# window rank 1 works
select activity
from
(select activity,
 rank() over (order by count(*) asc ) as cnt_asc,
 rank() over (order by count(*) desc) as cnt_desc
 from Friends
 group by activity) cte
where cnt_asc != 1 and cnt_desc != 1

# window rank 2 works
with cte as
(select
 activity,
 rank() over (order by count(id)) as counter
 from Friends
 group by activity
)
select
activity
from cte
where counter not in (select min(counter) from cte)
and   counter not in (select max(counter) from cte)
 
# 2 works 
select 
distinct activity
from Friends
group by activity
having count(activity) not in 
((select
count(*)
from Friends
group by activity
order by count(*) desc limit 1),
(select
count(*)
from Friends
group by activity
order by count(*) asc limit 1))


In [ ]:
# good question to try to answer using different techniques 
https://leetcode.com/problems/the-most-recent-three-orders/

# mine
with cte as     
(select
 order_id,
 order_date,
 customer_id,
 rank(customer_id) over (order by order_date desc) as ranking,
 cost
 from Orders
 group by customer_id
)
select
c.customer_name,
c.customer_id,
cte.order_id,
cte.order_date
from cte
join Customers c
on cte.customer_id = cte.customer_id
where cte.ranking <= 3;



# 2
SELECT a.name as customer_name,a.customer_id,b.order_id,b.order_date
FROM Customers as a
JOIN Orders as b
ON a.customer_id=b.customer_id
WHERE
(
    SELECT COUNT(*) FROM Orders as c
    WHERE b.customer_id=c.customer_id AND b.order_date<c.order_date
)<=2
ORDER BY customer_name,customer_id,order_date DESC;

# 1
select customer_name, customer_id, order_id, order_date
from
(select  name customer_name, c.customer_id, order_id, order_date,
rank() over(partition by c.customer_id order by order_date desc) rank
from customers c
inner join
orders o
on c.customer_id = o.customer_id) temp
where rank <=3
order by customer_name, customer_id, order_date desc

# 1
select
   customer_name,
   customer_id,
   order_id,
   order_date
from
(
    select
        name as customer_name,
        o.customer_id,
        order_id,
        order_date,
        dense_rank() over (partition by o.customer_id order by order_date desc) as rn
    from orders o
    join customers c
    on o.customer_id = c.customer_id
) subq
where rn < 4
order by
    customer_name,
    customer_id,
    order_date desc

    # 1
select name as customer_name, customer_id, order_id, order_date
from customers c
join (select customer_id, row_number()over (partition by customer_id order by order_date desc)row_num, order_id, order_date from orders) t
using(customer_id)
where row_num<=3
order by name, customer_id, order_date desc

# 1
select name as customer_name, customer_id, order_id, order_date
from
(select c.customer_id, c.name, o.order_id, o.order_date, row_number() over (partition by o.customer_id order by order_date desc) as ranking
from Customers c
join Orders o on c.customer_id = o.customer_id) t
where ranking <= 3
order by 1,2,4 desc

# 1
select customer_name,customer_id,order_id,order_date from
(select Name as customer_name,A.customer_id,A.order_id,A.order_date,dense_rank() over(partition by A.customer_id order by order_date desc) as rk from
Orders A left join Customers B
on A.customer_id =B.customer_id)x
where rk <= 3
order by customer_name,customer_id,order_date desc

# 1
select name as customer_name, customer_id,order_id, order_date from (
select name,order_id, b.customer_id, order_date, row_number() over (partition by b.customer_id order by order_date desc) as r from Customers a right join Orders b on a.customer_id=b.customer_id ) sub where r<=3 order by customer_name, customer_id,order_date desc

# 1
with cte_date as
(select customer_id, order_date, order_id, row_number() over(partition by customer_id order by order_date desc) as r
from Orders)

select name as customer_name, cte_date.customer_id, order_id, order_date
from cte_date
join Customers
on cte_date.customer_id = Customers.customer_id
where r <= 3
order by name, customer_id, order_date desc

# 1
with 
    customer_recent as (select customer_id, order_id, order_date, rank() over (partition by customer_id order by order_date desc) as weight  from orders)
    select customers.name as customer_name, customers.customer_id, customer_recent.order_id,  customer_recent.order_date from customer_recent join customers on customer_recent.customer_id = customers.customer_id where weight between 1 and 3 order by customer_name asc, customer_id asc, order_date desc
    

    


distinct activites
select distinct activities from Activities

# window rank 1 works
select activity
from
(select activity,
 rank() over (order by count(*) asc ) as cnt_asc,
 rank() over (order by count(*) desc) as cnt_desc
 from Friends
 group by activity) cte
where cnt_asc != 1 and cnt_desc != 1

# window rank 2 works
with cte as
(select
 activity,
 rank() over (order by count(id)) as counter
 from Friends
 group by activity
)
select
activity
from cte
where counter not in (select min(counter) from cte)
and   counter not in (select max(counter) from cte)
 
# 2 works 
select 
distinct activity
from Friends
group by activity
having count(activity) not in 
((select
count(*)
from Friends
group by activity
order by count(*) desc limit 1),
(select
count(*)
from Friends
group by activity
order by count(*) asc limit 1))
 

SELECT 
e.left_operand, 
e.operator, 
e.right_operand,
       CASE WHEN operator = '>' THEN IF(v1.value > v2.value, 'true', 'false')
            WHEN operator = '<' THEN IF(v1.value < v2.value, 'true', 'false')
            ELSE IF(v1.value = v2.value, 'true', 'false')
       END AS value
FROM Expressions e
JOIN Variables v1 ON v1.name = e.left_operand
JOIN Variables v2 ON v2.name = e.right_operand;    
    
    
1
select 
cte.project_id, 
cte.employee_id,
from
    (select 
     p.project_id,
     p.employee_id,
     rank() over(partition by p.project_id order by e.experience_years desc) as ranking
     from Project p join Employee e
     on p.employee_id = e.employee_id) cte
where cte.ranking = 1;





3
with cte as 
(select 
 project_id, 
 employee_id, 
 experience_years, 
 max(experience_years) over(partition by project_id) max_exp
 from project
 join employee using(employee_id))
select 
project_id, 
employee_id
from cte
where experience_years=max_exp




1 - get contacts of customers
2 - get contacts of customers that are in invoices

select 
user_id,
count(user_id) as contacts_cnt
from Contacts
group by user_id;


select 
user_id,
case when user_id in (select distinct user_id from Invoices) then count(user_id)
      else 0 end as  trusted_contacts_cnt
from Contacts
group by user_id;
 1, 3
2, 2
6, 1


select 
i.invoice_id,
c.customer_name,
i.price,
count(distinct t.contact_name) as contacts_cnt,
sum(case when contact_name in (select distinct customer_name from Customers) then 1 else 0 end) as trusted_contacts_cnt
from Invoices as i
join Customers as c on c.customer_id = i.user_id
left join Contacts as t on c.customer_id = t.user_id
group by i.invoice_id

select
i.invoice_id,
cu.customer_name,
i.price,
(select count(i.user_id) as contacts_cnt from Contacts group by user_id)
(select user_id,
 case when user_id in (select distinct user_id from Invoices) then count(user_id)
      else 0 end as  trusted_contacts_cnt
 from Contacts
 group by user_id)
from Invoices i
join Customers cu on cu.customer_id = i.user_id
join Contacts co on co.user_id = cu.customer_id
group by i.invoice_id
order by i.invoice_id



select 
i.invoice_id,
c.customer_name,
i.price ,
count(cont.contact_email) contacts_cnt ,
sum(if(cont.contact_name in (select distinct email from customers),1,0)) trusted_contacts_cnt
from invoices i
join customers c on c.customer_id = i.user_id
left join Contacts cont on cont.user_id = c.customer_id
group by i.invoice_id
order by i.invoice_id


WITH tax_multiplier AS (
SELECT company_id,
CASE WHEN MAX(salary) < 1000 THEN 1.0
WHEN MAX(salary) <= 10000 THEN 0.76
ELSE 0.51 END AS mult
FROM Salaries
GROUP BY company_id
)
SELECT s.company_id, s.employee_id, s.employee_name, ROUND(s.salary * t.mult) AS salary
FROM Salaries s JOIN tax_multiplier t ON s.company_id = t.company_id;



select 
s.company_id, 
s.employee_id, 
s.employee_name, 
round(s.salary * cte.factor) as salary
from Salaries s 
join 
 (select company_id,
  case when max(salary) < 1000 then 1.0
  when max(salary) <= 10000 then 0.76
  else 0.51 
  end as factor
  from Salaries
  group by 1) cte
using(company_id)



select
company_id,
employee_id,
employee_name,
case (when salary < 1000 then round(salary,0)
      when salary > 10000 then round(1.49*salary,0)
      else round(1.24*salary,0) end) as salary
from Salaries


select 
q.id, 
q.year, 
coalesce(n.npv, 0) as npv
from npv n 
right join queries q on n.id = q.id 
and n.year = q.year

select
distinct o.customer_id,
c.customer_name
from Orders o
where o.customer_id in 
(select distinct customer_id
 from orders 
 where product_name = 'A')
and o.customer_id in (select distinct customer_id
 from orders 
 where product_name = 'B')
and o.customer_id not in 
(select distinct customer_id
 from orders 
 where product_name = 'C')
inner join Customers c on c.customer_id = o.customer_id
order by o.customer_id 


SELECT 
customer_id, 
customer_name
FROM Customers
WHERE customer_id IN (
    SELECT customer_id FROM Orders
    GROUP BY customer_id
    HAVING SUM(product_name='A')>0 
        AND SUM(product_name='B')>0 
        AND SUM(product_name='C')=0
)
ORDER BY customer_id;


select
o.customer_id
from Orders o
where o.customer_id in 
(select distinct customer_id
 from orders 
 where product_name = 'A'
 or product_name = 'B')
and o.customer_id not in 
(select distinct customer_id
 from orders 
 where product_name = 'C');


select 
min(log_id) as start_id, 
max(log_id) as end_id
from(
 select log_id,
 log_id-row_number() over (order by log_id) rk1 
from logs) tmp
group by rk1


with cte as 
 (select 
  log_id
  log_id - row_number() over (order by log_id) as difference
  from logs)
select 
min(log_id) as start_id
max(log_id) as end_id
from cte
group by difference
order by start_id 

WITH temp1 AS(SELECT log_id
,log_id-ROW_NUMBER() OVER(ORDER BY log_id) AS difference
FROM Logs)

SELECT MIN(log_id) AS start_id
,MAX(log_id) AS end_id
FROM temp1
GROUP BY difference
ORDER BY start_id


select
gender,
day,
sum(score_points) over (partition by gender order by day) as total
from scores


selects 
s1.gender,
s1.day,
sum(s2.score_points) as total
from Scores s1
left join Scores s2 on s1.day >= s2.day and s1.gender = s2.gender


group by s1.gender, s1.day
order by s1.gender, s1.day 

group by gender, day
order by gender, day


select
sale_date
sum(case when fruit='apple' then sold_num
    case when fruit='orange' then -1*sold_num) as diff1
sum(if(fruit='apple'), sold_num, -1*sold_num)) as diff2
from sales
group by sale_date

select
sale_date
sum(apples-oranges) over (partition by sale_date) as diff

sold_num - lead(sold_num, 1) over (partition by sale_date) as diff

from Sales
order by sale_date


select
stock_name,
sum(case when operation='Buy' then 
    price - lag(price,1) over (partition by stock_name order by operation_day) end)
as capital_gain_loss
from Stocks
group by stock_name


select sale_date, sum(case when fruit='apples' then sold_num else -sold_num end) as diff
from sales
group by sale_date




with cte as
 (select *,
  case when operation='buy' then -1*price
  else price
  end as 'gain_loss'
  from Stocks)
select
stock_name,
sum(gain_loss) as 'capital_gain_loss'
from cte
group by stock_name
    
select
stock_name,
sum(case when operation='buy' then -1*price
    else price end) as capital_gain_loss
from stocks
group by stock_name
    
    
WITH TABLE1 AS (
    SELECT *, CASE WHEN operation = 'Buy' THEN price * -1 ELSE price END AS 'gain_loss'
    FROM Stocks
)
SELECT stock_name, SUM(gain_loss) AS 'capital_gain_loss'
FROM TABLE1
GROUP BY stock_name
;


select
abs(p1.x - p2.x) as shortest
from point p1
cross join point p2
where p1.x != p2.x
order by shortest asc limit 1

select
min(abs(p1.x - p2.x)) as shortest
from point p1
cross join point p2
where p1.x != p2.x



select
p1.x - p2.x
from point p1
cross join point p2
where p1.x != p2.x

select
name,
population,
area
from World
where area > 3000000
or population > 25000000;

select
o.customer_id,
c.name
from Customers c
join Orders o on o.customer_id = c.customer_id
having 

with 
(
    
) as cte

customer_id in 

with
(select
o.customer_id
p.price*o.quantity as sum1
from Orders o
join Product p on p.product_id = o.product_id
where order_date between '2020-06-01' and '2020-07-01'
and sum1 > 100
group by customer_id) cte1
and 
(select
o.customer_id
p.price*o.quantity as sum1
from Orders o
join Product p on p.product_id = o.product_id
where order_date between '2020-07-01' and '2020-08-01'
and sum1 > 100
group by customer_id) cte2
select 
c.customer_id,
c.name
from Customers c
where c.customer_id in cte1.customerid 
and c.customer_id in cte2.customerid




select
p.product_name
sum(o.unit) as unit
from Orders o
join Products p on o.product_id = p.product_id
where o.order_date between '2020-02-01' and '2020-03-01'
group by o.product_id
having unit > 100;

select distinct author_id as id
from Views
where author_id = viewer_id
order by id 


select
distinct v1.author_id as id
from views v1
inner join views v2 on v1.author_id = v2.viewer_id
order by id asc;

select
customer_number from
(select
customer_number,
count(*) as total_orders
from orders
group by customer_number
order by total_orders desc limit 1) cte

SELECT
    customer_number
FROM
    orders
GROUP BY customer_number
ORDER BY COUNT(*) DESC
LIMIT 1
;


select
customer_number
from orders
group by customer_number
order by count(*) desc limit 1

select 
customer_number 
from orders 
group by customer_number
having count(customer_number) >= all 
(select count(customer_number) from orders group by customer_number)


select
customer_number 
from
orders
select count(*) from orders group by customer
(select
customer_number,
count(*) as total_orders
from orders
group by customer_number
order by total_orders desc limit 1) cte




select
e.student_id,
s.student_name,
e.subject_name,
ifnull(count(*),0) as attended_exams
from Examinations e
left join Students s on s.student_id = e.student_id
group by e.student_id, e.subject_name
order by e.student_id, s.subject_name


"values": [[1, "Alice", "Math", 3], [1, "Alice", "Physics", 2], [1, "Alice", "Programming", 1], [2, "Bob", "Math", 1], [2, "Bob", "Programming", 1], [13, "John", "Math", 1], [13, "John", "Physics", 1], [13, "John", "Programming", 1]]}

"values": [[1,"Alice","Math",3],[1,"Alice","Physics",2],         [1,"Alice","Programming",1],[2,"Bob","Math",1],[2,"Bob","Physics",0],[2,"Bob","Programming",1],[6,"Alex","Math",0],[6,"Alex","Physics",0],[6,"Alex","Programming",0],[13,"John","Math",1],[13,"John","Physics",1],[13,"John","Programming",1]]}    
    
    
select
name
from customer
where referee_id != 2

select actor_id, 
director_id 
from ActorDirector
group by actor_id, director_id
having count(director_id) >= 3 

select
actor_id,
director_id
from ActorDirector
having count(*) > 3
group by actor_id, director_id

select
actor_id,
director_id
where actor_id, director_id in
(select actor_id, 
 director_id, 
 count(*) 
 from ActorDirector
 group by actor_id, director_id
 where count(*) >= 3)

ActorDirector



select
lower(trim(product_name)),
date_format(sale_date, '%Y-%m') as sale_date,
count(sale_id) as total
from sales
group by lower(trim(product_name)), date_format(sale_data, '%Y-%m')
order by product_name asc, sale_date asc;


select 
seller_id 
from Sales
group by seller_id
having sum(price) = (
    select sum(price) as total_price from Sales
    group by seller_id
    order by total_price desc
    limit 1
)

SELECT seller_id
FROM Sales
GROUP BY seller_id
HAVING SUM(PRICE) >= all (
    SELECT SUM(PRICE)
    FROM Sales
    GROUP BY seller_id
)



select
p.product_name,
s.price,
s.year
from Sales s
inner join Product p ON p.product_id = s.product_id;

SELECT DISTINCT 
    P.product_name, S.year, S.price 
FROM 
    (SELECT DISTINCT product_id, year, price FROM Sales) S
INNER JOIN
    Product AS P
USING (product_id);


SELECT 
p.product_name, d.year, d.price 
FROM Product as p 
JOIN 
  (SELECT DISTINCT(product_id), year, price FROM Sales) as d ON p.product_id = d.product_id


select
product_id,
sum(quantity) as total_quantity
from Sales
group by product_id
order by product_id;

coalesce(sum(r.distance),0) as travelled_distance


select
u.name,
coalesce(sum(r.distance),0) as travelled_distance
from Users u
join Rides r ON r.user_id = u.id
group by r.user_id
order by travelled_distance desc, u.name 


select
p.product_id,
round(sum(u.units*p.price)/sum(u.units),2) as average_price
from Prices p
join UnitsSold u 
ON p.product_id = u.product_id and (u.purchase_date > p.start_date) and (u.purchase_date <= p.end_date)
group by p.product_id

select 
player_id, 
device_id 
from activity 
where (player_id, event_date) in 
(select player_id,
        min(event_date)
        from activity 
        group by player_id) 

select
player_id,
device_id
from 
(select player_id, device_id, min(event_date) from Activity group by player_id) cte;


Activity
WHERE (player_id, event_date) IN
(SELECT player_id,
        MIN(event_date) 
        FROM Activity 
        GROUP BY player_id);



select
player_id,
device_id
from Activity
WHERE player_id, event_date =
(SELECT player_id,MIN(event_date) AS first_login FROM Activity GROUP BY player_id);

select id,
  sum(case when month = 'jan' then revenue else 0 end) as Jan_revenue
from department
group by id
order by id


select id, 
	sum(case when month = 'jan' then revenue else null end) as Jan_Revenue,
	sum(case when month = 'feb' then revenue else null end) as Feb_Revenue,
	sum(case when month = 'mar' then revenue else null end) as Mar_Revenue,
	sum(case when month = 'apr' then revenue else null end) as Apr_Revenue,
	sum(case when month = 'may' then revenue else null end) as May_Revenue,
	sum(case when month = 'jun' then revenue else null end) as Jun_Revenue,
	sum(case when month = 'jul' then revenue else null end) as Jul_Revenue,
	sum(case when month = 'aug' then revenue else null end) as Aug_Revenue,
	sum(case when month = 'sep' then revenue else null end) as Sep_Revenue,
	sum(case when month = 'oct' then revenue else null end) as Oct_Revenue,
	sum(case when month = 'nov' then revenue else null end) as Nov_Revenue,
	sum(case when month = 'dec' then revenue else null end) as Dec_Revenue
from department
group by id
order by id

select
e1.employee_id
 (SELECT 
  COUNT(e2.team_id) 
  FROM employee e2
  WHERE e1.team_id = e2.team_id
  GROUP BY team_id) as team_size
FROM employee e1

select e1.employee_id,(select count(e.team_id)
from employee e
where e.team_id=e1.team_id
group by team_id) as team_size
from employee e1


{"headers": ["CandidateID"], "values": [[2], [4], [3], [5]]}

SELECT
DISTINCT Num as ConsecutiveNums
FROM Logs
WHERE Num = LEAD(Num, 1) OVER (ORDER BY Id)
AND Num = LEAD(Num, 2) OVER (ORDER BY Id);


SELECT DISTINCT
    l1.Num AS ConsecutiveNums
FROM
    Logs l1,
    Logs l2,
    Logs l3
WHERE
    l1.Id = l2.Id-1
    AND l2.Id = l3.Id-1
    AND l1.Num = l2.Num
    AND l2.Num = l3.Num
;


SELECT 
DISTINCT num
FROM
 (
  SELECT num,
  LEAD(num,1) OVER(ORDER BY id) AS lead,
  LAG(num,1) OVER (ORDER BY id) AS lag
  FROM logs
  ) cte
WHERE num=lead and num=lag;


SELECT
c.name
FROM Candidate c
JOIN
(SELECT CanddateID,
 DENSE_RANK() OVER (ORDER BY count(CandidateID) DESC) as ranking
 FROM Vote) cte
ON c.id = cte.candidateid
WHERE cte.ranking = 1


select C.Name from
Candidate C
JOIN(
select CandidateId, dense_rank() OVER (ORDER BY count(CandidateId) desc) as rnk from Vote
GROUP BY CandidateId)Voting
ON C.id = Voting.CandidateId
WHERE Voting.rnk=1


SELECT
name
FROM 
(SELECT
 name
 DENSE_RANK() OVER (PARTITION BY CandidateID ORDER BY count(CandidateId) DESC) AS ranking
 FROM Candidate C
 INNER JOIN Vote v ON c.id = v.cadidateid
 GROUP BY name) cte
where cte.ranking = 1

SELECT 
c.name
FROM Candidate c
JOIN
(SELECT
 CandidateID as id
 FROM Vote
 GROUP BY Canddateid
 ORDER BY COUNT(*) DESC
 LIMIT 1) AS cte
WHERE c.id = cte.id


SELECT
name
FROM Candidate
WHERE ID = 
(SELECT CandidateID
 FROM Vote
 GROUP BY CandidateID
 ORDER BY COUNT(*) DESC
 LIMIT 1)




SELECT
c.Name
FROM Candidate c
JOIN Vote v ON v.CandidateID = c.id
HAVING COUNT (v.CandidateID) >= ANY (
  SELECT COUNT(CandidateID) 
  FROM Vote 
  GROUP BY CandidateID))

SELECT
Score as score,
DENSE_RANK() OVER (ORDER BY Score DESC) AS Rank
FROM Scores;


SELECT
d.Name AS Department,
e.Employee as Employee,
e.Salary
FROM 
(SELECT e.*
 DENSE_RANK() OVER (PARTITION BY DepartmentID ORDER BY Salary DESC) AS DeptPayRank
 FROM Employee e
) cte
JOIN Department d on d.Id = cte.DepartmentId
WHERE DeptPayRank <= 3;

RANK salary OVER (PARTITION BY e.DepartmentID ORDER BY e.Salary DESC)
FROM Employee e
LIMIT 3;



select 
d.Name as Department, 
a.Name as Employee, 
a.Salary 
from (
  select e.*,
  dense_rank() over (partition by DepartmentId order by Salary desc) as DeptPayRank 
from Employee e 
) a 
join Department d
on a.DepartmentId = d. Id 
where DeptPayRank <=3; 

SELECT 
a.Department, 
a.Employee, 
a.Salary FROM
  (Select 
   b.Name as Department,
   a.Name as Employee,
   a.salary As Salary,
   DENSE_RANK() over (PARTITION BY b.Name ORDER BY a.Salary DESC)  as dept_sal_rank
from Employee a
INNER JOIN Department b ON a.DepartmentId = b.Id ) a
where a.dept_sal_Rank <=3

select 
d.Name Department, 
e1.Name Employee, 
e1.Salary
from Employee e1 
join Department d
on e1.DepartmentId = d.Id
where 
 (select count(distinct(e2.Salary)) 
  from Employee e2 
  where e2.Salary > e1.Salary 
  and e1.DepartmentId = e2.DepartmentId) < 3;


    





SELECT 
DISTINCT Salary 
FROM Employee 
ORDER BY Salary DESC 
LIMIT N-1, 1

SELECT
SALARY 
as CONCAT('genNthHighestSalary(',N,')')'
FROM Employee
ORDER BY Salary DESC
LIMIT 1 OFFSET N

select 
distinct Salary 
from Employee 
order by Salary desc 
limit N,1


SELECT DISTINCT Salary 
FROM Employee ORDER BY Salary DESC
LIMIT N-1 ,1


SELECT CASE 
  WHEN Grade >= 8 THEN Name
  ELSE "NULL" END, 
  Grade, 
  Marks 
FROM Students, Grades 
WHERE Students.Marks >= Min_Mark AND Students.Marks <= Max_Mark 
ORDER BY Grade DESC, Name


SELECT
IF (Grade >= 8, Name, NULL),
Grade,
Marks
FROM
  (SELECT Name,
  (SELECT Grade FROM Grades WHERE (Min_Mark <= Marks) AND (Marks <= Max_Mark)) AS Grade,
  Marks
  FROM Students) As MyStudents
ORDER BY Grade DESC, Name;



SELECT
LAT_N
FROM STATION
LIMIT 1 OFFSET 1
WHERE LAT_N > 
(SELECT AVG(LAT_N) FROM STATION); 
ORDER BY LAT_N ASC;

SELECT
LAT_N
FROM STATION
LIMIT 1 OFFSET 1
WHERE LAT_N > 
(SELECT AVG(LAT_N) FROM STATION); 
ORDER BY LAT_N ASC;




c.company_code,
c.founder,
COUNT(DISTINCT l.lead_manager_code), 
COUNT(DISTINCT s.senior_manager_code), 
COUNT(DISTINCT m.manager_code), 
COUNT(DISTINCT e.employee_code)
FROM Company c
JOIN Lead_Manager l ON c.company_code = l.company_code
JOIN Senior_Manager s ON c.company_code = s.company_code
JOIN Manager m ON c.company_code = m.company_code
JOIN Employee e ON c.company_code = e.company_code
ORDER BY c.company_code;

select company_code, founder,
(select count(distinct lead_manager_code) from Lead_Manager where company_code=c.company_code),
(select count(distinct senior_manager_code) from Senior_Manager where company_code=c.company_code),
(select count(distinct manager_code) from Manager where company_code=c.company_code),
(select count(distinct employee_code) from Employee where company_code=c.company_code)
from Company c
order by company_code;

SELECT
c.company_code,
c.founder,
COUNT(DISTINCT l.lead_manager_code), 
COUNT(DISTINCT s.senior_manager_code), 
COUNT(DISTINCT m.manager_code), 
COUNT(DISTINCT e.employee_code)
FROM Company c
JOIN Lead_Manager l ON c.company_code = l.company_code
JOIN Senior_Manager s ON c.company_code = s.company_code
JOIN Manager m ON c.company_code = m.company_code
JOIN Employee e ON c.company_code = e.company_code
ORDER BY c.company_code;



company_code, 
founder name, 
total number of lead managers, 
total number of senior managers, 
total number of managers, 
and total number of employees
Order your output by ascending company_code.



SELECT 
name 
FROM Employee
WHERE salary > 2000 
AND months < 10
ORDER BY employee_id;

SELECT N,
CASE (WHEN P IS NULL THEN 'Root' END) AS type
FROM BST
UNION
SELECT N,
CASE (WHEN N IN (SELECT DISTINCT P FROM BST WHERE P IS NOT NULL) THEN 'Inner'
      ELSE 'Leaf' END) AS type
FROM BST 
ORDER BY N;

SELECT CASE
  (WHEN P IS NULL THEN CONCAT(N, ' Root')
   WHEN N IN (SELECT DISTINCT P FROM BST) THEN CONCAT(N, ' Inner')
   ELSE CONCAT(N, ' Leaf')
   END)
FROM BST
ORDER BY N ASC;


select concat(name, '(', substring(occupation, 1, 1), ')') 
from occupations order by name asc;
select concat("There are total ", cast(count(*) as char), " ", lower(occupation), "s.") 
from occupations group by occupation order by count(*) asc;


Of sales that had a valid promotion, 
the VP of marketing wants to know what % of transactions 
occur on either the very first day or the
very last day of a promotion campaign.

select 
100.0*(count sa.transaction_date)
FROM sales as
join promotions pr ON sa.promotion_id = pr.promotion_id
where pr.promotion_id is not null 
and  (sa.transaction_date = pr.start_end
or     sa.transcation_date = pr.end_data)
/ (select count(transaction_date) from sales where promotion_id is not null);

select
100.0 * SUM(CASE WHEN tr.transaction_date = pr.start_date THEN 1
         WHEN tr.transaction_data = pr.end_date THEN 1
         ELSE 0 END)
/ (count(sa.transaction_date))
FROM sales sa
join promotions pr ON sa.promotion_id = pr.promotion_id
where pr.promotion_id is not null 

select 
100.0*sum(case when s.transaction_date = p.start_date then 1
 when s.transaction_date = p.end_date then 1
 else 0 end) /
 count(s.*) as pct_of_transactions_on_first_or_last_day_of_valid_promotion
from sales s
inner join promotions p
 on p.promotion_id = s.promotion_id
where s.promotion_id <> 0


from sales where promotion_id is not null);



with

as cte
select 100.0*
(select sales_cte FROM cte)/
/ (select count(transaction_date) from sales where promotion_id is not null);


FROM promotions pr
ORDER BY     DESC
LIMIT 5

select
100.0*
SUM(CASE WHEN islowfat+is_recyclable=2 THEN 1 ELSE 0 END) / 
COUNT( distinct product_id)
from products 


select
 sum(case when is_low_fat_flg + is_recyclable_flg = 2 then 1
 else 0 end) * 100.0 /
 count(distinct product_id) as pct_low_fat_and_recyclable
from products;


select
product_name
from product
WHERE ID = ANY
(SELECT product_id FROM orderitem WHERE quantity = 1)

select
id
from customer
where 

select
name
from customer
where id = ANY
(select distinct customerid FROM order where total_amount > avg(total_amount))

select
p.product_id,
round(sum(u.units*p.price)/sum(u.units),2) as average_price
from Prices p
join UnitsSold u 
ON p.product_id = u.product_id and (u.purchase_date between p.start_date and p.end_date)
group by p.product_id



select
round(
    ifnull(
    (select count(*) from (select distinct requester_id, accepter_id from request_accepted) as A)
    /
    (select count(*) from (select distinct sender_id, send_to_id from friend_request) as B),
    0)
, 2) as accept_rate;


(select count(*) from (select distinct requester_id, accepter_id from request_accepted) as A)


select round(
(select 
count(distinct requester_id, accepter_id )
from request_accepted) /
(select
count(distinct sender_id, send_to_id)
from friend_request),2)
as accept_rate;

SELECT
p.product_name,
SUM(o.unit) as unit
FROM Products p
JOIN Orders o ON o.product_id = p.product_id
WHERE order_date >= '2020-02-01'
AND order_date < '2020-03-01'
GROUP BY p.product_id
HAVING unit > 100;

[["Leetcode Solutions", 130], ["Jewels of Stringology", 80], ["HP", 5], ["Leetcode Kit", 100]]}

select 
b.book_id, 
b.name
from books b left join
    (select book_id, sum(quantity) as book_sold
    from Orders 
    where dispatch_date between '2018-06-23' and '2019-06-23'
    group by book_id) t
on b.book_id = t.book_id
where available_from < '2019-05-23'
and (book_sold is null or book_sold <10)
order by b.book_id;

select b.book_id, b.name from
(select * from books where available_from < '2019-05-23') b
left join
(select * from Orders where dispatch_date > '2018-06-23') o
on b.book_id = o.book_id
group by b.book_id, b.name
having sum(o.quantity) is null or sum(o.quantity) <10;

select b.book_id, b.name from
(select * from books where available_from < '2019-05-23') b
left join
(select * from Orders where dispatch_date > '2018-06-23') o
on b.book_id = o.book_id
group by b.book_id, b.name
having sum(o.quantity) is null or sum(o.quantity) <10;


[[1, "Kalila And Demna", 2], [3, "The Hobbit", 1], [4, "13 Reasons Why", 2], [5, "The Hunger Games", 2]]}
[[1,"Kalila And Demna"],[2,"28 Letters"],[5,"The Hunger Games"]]}

SELECT
ROUND(
  (SUM(CASE WHEN order_date = customer_pref_delivery_date THEN 1 ELSE 0 END) / COUNT(delivery_id)),2) 
  AS immediate_percentage
FROM Delivery;

SELECT
(CASE WHEN order_date = customer_pref_delivery_date THEN 1 ELSE 0 END)
AS immediate_percentage
FROM Delivery;


SELECT 
parent_id AS post_id,
COUNT(sub_id, parent_id) AS number_of_comments
FROM Submissions
WHERE parent_id IS NOT NULL
AND parent_id IN 
(SELECT DISTINCT sub_id from Submissions WHERE parent_id IS NULL)
GROUP BY parent_id
ORDER BY number_of_comments DESC;


select 
a.sub_id as post_id, 
count(distinct b.sub_id) as number_of_comments
from Submissions a left join Submissions b on a.sub_id = b.parent_id 
where a.parent_id is null
group by a.sub_id

SELECT 
s.sub_id AS post_id,
(SELECT COUNT(DISTINCT(s1.sub_id)) FROM Submissions s1 WHERE s1.parent_id = s.sub_id) 
AS number_of_comments
FROM Submissions s
WHERE s.parent_id IS null
GROUP BY s.sub_id


{"headers": ["post_id", "number_of_comments"], "values": [
    [null, 0], 
    [1, 4], 
    [2, 2], 
    
    
{"headers":["post_id","number_of_comments"],"values":[
    [1,3],
    [2,2],
    [12,0]]}
